In [ ]:
# TCN 18 MONTHS BTC DATA WITH SENTIMENT
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN  # Ensure you have installed via: pip install keras-tcn


# Split merged data into training and testing sets based on date ranges
train_df = merged_df[(merged_df['datetime'] >= '2018-01-01') & (merged_df['datetime'] < '2019-02-01')].copy()
test_df = merged_df[merged_df['datetime'] >= '2019-02-01'].copy()

# Scale the features separately using MinMaxScaler
close_scaler = MinMaxScaler()
volume_scaler = MinMaxScaler()
sentiment_scaler = MinMaxScaler()

train_df['close_scaled'] = close_scaler.fit_transform(train_df[['close']])
train_df['volume_scaled'] = volume_scaler.fit_transform(train_df[['volume']])
train_df['sentiment_scaled'] = sentiment_scaler.fit_transform(train_df[['avg_sentiment']])
test_df['close_scaled'] = close_scaler.transform(test_df[['close']])
test_df['volume_scaled'] = volume_scaler.transform(test_df[['volume']])
test_df['sentiment_scaled'] = sentiment_scaler.transform(test_df[['avg_sentiment']])

# Create numpy arrays with three features in proper order: [close_scaled, volume_scaled, sentiment_scaled]
train_scaled = train_df[['close_scaled', 'volume_scaled', 'sentiment_scaled']].values
test_scaled = test_df[['close_scaled', 'volume_scaled', 'sentiment_scaled']].values

# ----------------------------
# 2. Creating Sequences for Time Series
# ----------------------------
# Define the lookback period; ensure that the sequences respect the temporal order
lookback = 30  # Use previous 30 days to predict the next day's close

def create_sequences(data, lookback):
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i - lookback:i])  # Each sequence is of shape (lookback, 3)
        y.append(data[i, 0])  # Target remains the close price (first column)
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ----------------------------
# 3. Building and Training the TCN Model
# ----------------------------
# Build the TCN model using three input features (close, volume, and sentiment)
model = Sequential()
model.add(TCN(nb_filters=64, kernel_size=3, dropout_rate=0.1, return_sequences=True, input_shape=(lookback, 3)))
model.add(TCN(nb_filters=64, kernel_size=3, dropout_rate=0.1, return_sequences=False))
model.add(Dense(1))  # Predict the close price
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1)

# ----------------------------
# 4. Prediction and Inverse Transformation
# ----------------------------
predictions_scaled = model.predict(X_test)
prediction_tcn = close_scaler.inverse_transform(predictions_scaled)
y_test_actual_tcn = close_scaler.inverse_transform(y_test.reshape(-1, 1))

# Align the dates (since we lose the first 'lookback' days in sequence creation)
test_dates = test_df['datetime'].values[lookback:]

# ----------------------------
# 5. Visualization: Predicted vs Actual Close Prices
# ----------------------------
plt.figure(figsize=(12, 6))
plt.plot(test_dates, y_test_actual_tcn, label='Actual Close Price', color='blue')
plt.plot(test_dates, prediction_tcn, label='Predicted Close Price (TCN with Volume & Sentiment)', color='red', linestyle='--')
plt.xlabel('Date')
plt.ylabel('BTC Close Price')
plt.title('BTC Close Price Prediction: Actual vs Predicted (Mar 15 - May 1, 2019)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# ----------------------------
# 6. Performance Metrics
# ----------------------------
mse = mean_squared_error(y_test_actual_tcn, prediction_tcn)
mae = mean_absolute_error(y_test_actual_tcn, prediction_tcn)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test_actual_tcn, prediction_tcn) * 100
r2 = r2_score(y_test_actual_tcn, prediction_tcn)

print("Performance Metrics for the TCN Model (Including Volume & Sentiment):")
print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")
print(f"R2 Score: {r2:.2f}")
